In [1]:
from utils import normalize_df, create_train_test_val_df
import os
notebook_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
data_dir = os.path.join(root_dir, 'data')
import numpy as np
import pandas as pd
from sklearn.metrics import brier_score_loss, make_scorer, log_loss, mean_squared_error
from IPython.display import display_html
from copy import deepcopy
import pickle
from sklearn import calibration
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GroupKFold, RandomizedSearchCV, cross_val_predict, GridSearchCV
# from utils.utils

# Model 1.0
This notebook will act as an interactive tutorial for our Live Win Probability Model. This "model" is actual comprised of 3 separate models that "stack" on each other.
1. Play and drive outcome models
    * technically this is two separate models:
        * Play outcome (first down, field goal made, field goal missed, touchdown, turnover, and none/other)
            * only using the first down prediction from the output of this model
        * Drive outcome (Clock, field goal made, field goal missed, punt, safety, touch down, turnover, turnover on downs)
    * outputs for both models will be a series of probabilities for each class that all add up to 1
2. End of regulation score differential model
    * Dealing with overtime later, we want to predict how the score differential will change by the end of regulation.
        * i.e., if the current score differential (home score - away score) is -3 and the end of regulation score differential is -10, the target value will be -7
    * Output of this will be a series of probabilities from for all score differential possibilities from -35 to 35 (outputs <-35 or >35 will be set to -35/35 respectively)
3. End of regulation score total model
    * Similar concept to the score differential model
    * Again, we're using the change in end of regulation score total as the target value
    * Outputs will be a series of probabilites for classes from 0 to 83 (outputs will be capped at 83)

## Data
Let's take a look at the data that we are pulling from oracle
* First we have event_df and odds_df
* event_df is the play by play data mixed with some import game information
* Odds data has vegas predictions for almost all the games in the set (missing games will be given the average vegas spread and over/under)
    * The spread and over/under are merged with the event table to give us our pre-game priors
    * some games have multiple odds so duplicates are removed


In [2]:
event_df = pd.read_parquet(os.path.join(data_dir, "event_data.parquet"))
event_df = event_df.drop_duplicates(["nevent", "game_code"]).reset_index(drop=True)
odds_df = pd.read_parquet(os.path.join(data_dir, "odds_data.parquet"))
odds_df = odds_df.drop_duplicates("game_code")
event_df[["cur_spread", "cur_over_under"]] = event_df.merge(odds_df, how="left", on="game_code")[["cur_spread", "cur_over_under"]].fillna({"cur_spread": np.mean(odds_df["cur_spread"]), "cur_over_under": np.mean(odds_df["cur_over_under"])})
pd.set_option("display.max_columns", None)
display_html(event_df)

,game_code,game_date,season,home_team_id,home_team,home_team_abbrev,away_team_id,away_team,away_team_abbrev,home_final_score,away_final_score,final_score_diff,end_of_regulation_score_diff,home_rest_of_game_score,away_rest_of_game_score,end_of_regulation_score_diff_change,home_score_added,away_score_added,current_score_diff,current_score_total,home_start_score,away_start_score,home_team_outcome,home_team_win,draw,away_team_win,nevent,quarter,overtime,home_team_has_ball,off_team_id,def_team_id,kick_off,punt,point_after_kick,two_point_attempt,field_goal_attempt,off_start_score,off_end_score,off_score_change,def_start_score,def_end_score,def_score_change,play_counts,efficiency_counts,from_scrimmage,first_down,scoring_play,possession_change,continuation,event_name,event_id,yards_gained,drive_outcome_id,drive_outcome_desc,down,ytg,yd_from_goal,drive_id,drive_start,play_start_time,cur_spread,cur_over_under
0,819846,2008-09-04,2008,351,New York Giants,NYG,363,Washington Redskins,Was,16,7,9,9,16,7,9,0,0,0,0,0,0,W,1,0,0,1,1,0,0,363,351,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,Kick Off,5,73.0,NaN,None,0,-1,70,NaN,3600,900.0,-4.5,41.5
1,819846,2008-09-04,2008,351,New York Giants,NYG,363,Washington Redskins,Was,16,7,9,9,16,7,9,0,0,0,0,0,0,W,1,0,0,2,1,0,1,351,363,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,Kick Off Return,6,19.0,NaN,None,0,-1,103,NaN,3600,900.0,-4.5,41.5
2,819846,2008-09-04,2008,351,New York Giants,NYG,363,Washington Redskins,Was,16,7,9,9,16,7,9,0,0,0,0,0,0,W,1,0,0,3,1,0,1,351,363,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,Run,4,3.0,37.0,TD,1,10,84,1.0,3600,895.0,-4.5,41.5
3,819846,2008-09-04,2008,351,New York Giants,NYG,363,Washington Redskins,Was,16,7,9,9,16,7,9,0,0,0,0,0,0,W,1,0,0,4,1,0,1,351,363,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,Incomplete Pass,2,0.0,37.0,TD,2,7,81,1.0,3600,860.0,-4.5,41.5
4,819846,2008-09-04,2008,351,New York Giants,NYG,363,Washington Redskins,Was,16,7,9,9,16,7,9,0,0,0,0,0,0,W,1,0,0,5,1,0,1,351,363,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,Pass Completion,1,8.0,37.0,TD,3,7,81,1.0,3600,854.0,-4.5,41.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816793,2337728,2022-02-13,2021,327,Cincinnati Bengals,Cin,343,Los Angeles Rams,LAR,20,23,-3,-3,0,0,0,0,0,-3,43,20,23,L,0,0,1,201,4,0,1,327,343,0,0,0,0,0,20,20,0,23,23,0,1,1,1,0,0,0,0,Run,4,0.0,40.0,Downs,3,1,49,13.0,85,48.0,4.5,48.5
816794,2337728,2022-02-13,2021,327,Cincinnati Bengals,Cin,343,Los Angeles Rams,LAR,20,23,-3,-3,0,0,0,0,0,-3,43,20,23,L,0,0,1,202,4,0,1,327,343,0,0,0,0,0,20,20,0,23,23,0,1,0,0,0,0,0,0,Offense Timeout,57,NaN,40.0,Downs,4,1,49,13.0,85,43.0,4.5,48.5
816795,2337728,2022-02-13,2021,327,Cincinnati Bengals,Cin,343,Los Angeles Rams,LAR,20,23,-3,-3,0,0,0,0,0,-3,43,20,23,L,0,0,1,203,4,0,1,327,343,0,0,0,0,0,20,20,0,23,23,0,1,1,1,0,0,1,0,Incomplete Pass,2,0.0,40.0,Downs,4,1,49,13.0,85,43.0,4.5,48.5
816796,2337728,2022-02-13,2021,327,Cincinnati Bengals,Cin,343,Los Angeles Rams,LAR,20,23,-3,-3,0,0,0,0,0,-3,43,20,23,L,0,0,1,204,4,0,0,343,327,0,0,0,0,0,23,23,0,20,20,0,1,1,1,0,0,0,0,Run,4,-1.0,39.0,End Game,1,10,51,13.0,39,39.0,4.5,48.5


Adding timeouts remaining for both teams and time left in game

In [3]:
event_df["half"] = round((event_df["quarter"] + 0.01) / 2)
event_df["home_timeout"] = np.where(((event_df["event_id"]==57)&(event_df["home_team_has_ball"]==1))|((event_df["event_id"]==58)&(event_df["home_team_has_ball"]==0)), 1, 0)
event_df["away_timeout"] = np.where(((event_df["event_id"]==57)&(event_df["home_team_has_ball"]==0))|((event_df["event_id"]==58)&(event_df["home_team_has_ball"]==1)), 1, 0)
event_df["home_timeouts_remaining"] = np.clip(3 - event_df.groupby(["game_code", "half"])["home_timeout"].cumsum(), 0, 3)
event_df["away_timeouts_remaining"] = np.clip(3 - event_df.groupby(["game_code", "half"])["away_timeout"].cumsum(), 0, 3)
event_df["time_left_in_game"] = np.where(event_df["quarter"] <= 4, event_df["play_start_time"] + (4 - event_df["quarter"]) * 900, event_df["play_start_time"])
# event_df["time_elapsed"] = 900 - event_df["play_start_time"] + (event_df["quarter"] - 1) * 900


* Our PBP will have multiple rows for one play, so if there's a fumble then recovery by offense and a touchdown, 
* that could have 2-3 rows of data and the touchdown wouldn't show up as being apart of the original play 
    * plays would look like this: 1. Run, 2. Fumble, 3. Offense Recovers the ball (TD)
* So what we've done here is ensure that plays that are "continuation" that end in a touchdown, give a TD=True for all of the plays in the sequence
* After that is taken care of we can setup all of the labels for play and drive description

In [4]:
event_df["sequence"] = event_df["continuation"].groupby(event_df["continuation"].eq(0).cumsum()).cumsum()
event_df["play_start_id"] = event_df["nevent"] - event_df["sequence"]

turnover_ids = [9, 16]
event_df["turnover"] = np.where(event_df["event_id"].isin(turnover_ids), 1, 0)
event_df["touchdown_scored"] = np.where(event_df["home_score_added"]+event_df["away_score_added"]>=6, 1, 0)
event_df["fieldgoal_made"] = np.where(event_df["home_score_added"]+event_df["away_score_added"]==3, 1, 0)

play_outcome_aggregate =event_df[["game_code", "play_start_id", "turnover", "touchdown_scored", "fieldgoal_made", "first_down"]].groupby(["game_code", "play_start_id"], as_index=False).sum()
event_df["touchdown_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["touchdown_scored_y"], 0, 1)
event_df["turnover_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["turnover_y"], 0, 1)
event_df["field_goal_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["fieldgoal_made_y"], 0, 1)
event_df["first_down_in_play"] = np.clip(event_df.merge(play_outcome_aggregate,on=["game_code", "play_start_id"], how="left")["first_down_y"], 0, 1)



event_df["play_outcome"] = (
    np.where((event_df["turnover_in_play"]==1), "turnover",
    # (event_df["touchdown_in_play"]==1)&(event_df["turnover_in_play"]==1), "defensive_touchdown", 
    np.where((event_df["punt"]==1), "punt",
    np.where((event_df["field_goal_in_play"]==1), "field_goal_made",
    np.where((event_df["field_goal_attempt"]==1)&(event_df["field_goal_in_play"]==0), "field_goal_missed",
    np.where((event_df["first_down_in_play"]==1)&(event_df["touchdown_in_play"]==0)&(event_df["turnover_in_play"]==0)&(event_df["punt"]==0), "first_down",
    np.where((event_df["touchdown_in_play"]==1)&(event_df["turnover_in_play"]==0), "offensive_touchdown", "none"))))))
)
drive_description_matrix = {
    7: "punt",
    9: "turnover",
    14: "turnover",
    17: "field_goal_made",
    18: "punt",
    20: "safety",
    35: "field_goal_missed",
    36: "field_goal_missed",
    37: "touch_down",
    38: "clock",
    39: "clock",
    40: "turnover_on_downs",
    42: "field_goal_made",
    51: "clock",
}
event_df["drive_outcome_desc_basic"] = event_df["drive_outcome_id"].map(drive_description_matrix)

# event_df["drive_outcome"] = np.where(
#     (event_df["touchdown_in_drive"]==1)&(event_df["turnover_in_drive"]==1), "defensive_touchdown", 
#     np.where((event_df["touchdown_in_drive"]==1)&(event_df["turnover_in_drive"]==0), "offensive_touchdown",
#     np.where((event_df["field_goal_in_drive"]==1), "field_goal_made",
#     np.where((event_df["touchdown_in_drive"]==0)&(event_df["turnover_in_drive"]==1), "turnover", "none"
# ))))
game_end_of_regulation_total_score = event_df[event_df.overtime==0].groupby("game_code", as_index=False).max()[["game_code", "home_start_score", "away_start_score"]]
game_end_of_regulation_total_score["end_of_regulation_score_total"] = game_end_of_regulation_total_score["home_start_score"] + game_end_of_regulation_total_score["away_start_score"]
# event_df["end_of_regulation_score_total_diff"] = 
event_df["end_of_regulation_score_total_diff"] = (
    event_df.merge(game_end_of_regulation_total_score, on="game_code")["end_of_regulation_score_total"]
    - (event_df["home_start_score"] + event_df["away_start_score"])
)

/var/folders/dq/tcxv3s4j7y585_4jhfbdvrx00000gn/T/ipykernel_2915/1115593539.py:50: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  game_end_of_regulation_total_score = event_df[event_df.overtime==0].groupby("game_code", as_index=False).max()[["game_code", "home_start_score", "away_start_score"]]


### Data Manipulation
* We need to do a little bit of data manipulation to get the values we need, but we don't want to "overwrite" the values in event_df so we'll make copy of it called model_df
* time left in half is added
* from_scrimmage is changed so that PATs and two point conversions are not included
* down, ytg, and yd_from_goal are changed so that all non-scrimmage plays are changed to a default "null" value
* home_team_has_ball is change so that when kickoffs occur, the team receiving is the one that is in possession of the ball

### Data Subset
* Removing continuation plays that we mentioned before, so that each snap has just one target
* Remove plays where the down is equal to 0 
* Remove plays from scrimmage that did not count (e.g., plays that were waved off by penalties)
* scrimmage_plays_we_want is event_id of all the scrimmage plays that *aren't* timeouts, end of quarters, and the two minute warning.
* Remove all NA values for the feature inputs and target
* Remove all plays that are not from scrimmage
* Remove all overtime plays

In [5]:
model_df = deepcopy(event_df)
model_df["time_left_in_half"] = event_df["time_left_in_game"] - ((2 - event_df["half"]) * 1800)
model_df["from_scrimmage"] = np.where(event_df["event_id"].isin([22, 47, 52, 53, 54, 55, 56]), 0, event_df["from_scrimmage"])
model_df["down"] = np.where(model_df["from_scrimmage"] == 0, 0, event_df["down"])
model_df["ytg"] = np.where(model_df["from_scrimmage"] == 0, -1, event_df["ytg"])
model_df["yd_from_goal"] = np.where(model_df["from_scrimmage"] == 0, -1, event_df["yd_from_goal"])
model_df["home_team_has_ball"] = np.where(event_df["event_id"].isin([5]), 1 - event_df["home_team_has_ball"], event_df["home_team_has_ball"])
scrimmage_plays_we_want = [1, 2, 3, 4, 7, 9, 14, 17, 18, 35]


In [7]:

input_names = [
    'time_left_in_half',
    'half',
    'current_score_diff',
    'current_score_total',
    'cur_spread',
    'cur_over_under',
    'home_timeouts_remaining',
    'away_timeouts_remaining',
    # 'punt',
    # 'field_goal_attempt',
    'ytg',
    'yd_from_goal',
    'down',
    'home_team_has_ball',
]
output_name = "play_outcome"
mask_model = (
    (model_df.continuation==0)&
    (model_df.down!=0)&
    (model_df.play_counts==1)&
    (model_df.event_id.isin(scrimmage_plays_we_want))&
    (model_df[input_names+[output_name]].notna().all(axis=1))&
    (model_df["from_scrimmage"]==1)&
    (model_df["overtime"]==0)
)
X_train, y_train, group_train, X_test, y_test, group_test, X_val, y_val, group_val = create_train_test_val_df(model_df[mask_model], input_names, output_name)


Let's take a look at what the input features and output features look like. 

Event Name and yards gained is included to help interpret what is going on. 

This is the first 2 drives of the first game in the dataset

In [8]:
model_df[mask_model][input_names + ["event_name", "yards_gained", "play_outcome", "drive_outcome_desc_basic"]].head(15)

,time_left_in_half,half,current_score_diff,current_score_total,cur_spread,cur_over_under,home_timeouts_remaining,away_timeouts_remaining,ytg,yd_from_goal,down,home_team_has_ball,event_name,yards_gained,play_outcome,drive_outcome_desc_basic
2,1795.0,1.0,0,0,-4.5,41.5,3,3,10,84,1,1,Run,3.0,none,touch_down
3,1760.0,1.0,0,0,-4.5,41.5,3,3,7,81,2,1,Incomplete Pass,0.0,none,touch_down
4,1754.0,1.0,0,0,-4.5,41.5,3,3,7,81,3,1,Pass Completion,8.0,first_down,touch_down
5,1723.0,1.0,0,0,-4.5,41.5,3,3,10,73,1,1,Run,3.0,none,touch_down
6,1676.0,1.0,0,0,-4.5,41.5,3,3,7,70,2,1,Pass Completion,30.0,first_down,touch_down
7,1650.0,1.0,0,0,-4.5,41.5,3,3,10,40,1,1,Pass Completion,19.0,first_down,touch_down
8,1612.0,1.0,0,0,-4.5,41.5,3,3,10,21,1,1,Run,4.0,none,touch_down
9,1562.0,1.0,0,0,-4.5,41.5,3,3,6,17,2,1,Incomplete Pass,0.0,none,touch_down
10,1555.0,1.0,0,0,-4.5,41.5,3,3,6,17,3,1,Pass Completion,11.0,first_down,touch_down
11,1517.0,1.0,0,0,-4.5,41.5,3,3,6,6,1,1,Incomplete Pass,0.0,none,touch_down


### Loading in the Models and Creating Play/Drive Predictions
* ~~For this exercise we won't be training the models, just loading saved models and then using them to make predictions~~
* Now we are going to train the models
* In addition, each prediction will be split up between home and away. So if the home team has the ball the predictions for the away team play/drive outcomes are going to be set to 0


Let's take a look at how the predictions look for our dataset

In [9]:
cv=GroupKFold(n_splits=3)

rf_grid = {
    "n_estimators": np.linspace(start=50, stop=500, num=10, dtype=int),
    "max_features": ["auto", "sqrt"],
    "max_depth": np.linspace(5, 15, num=11, dtype=int),
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True, False],
}

log_loss_scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
rf_play_outcome_search_model = RandomForestClassifier(verbose=100, n_jobs=-1, random_state=1)
search_rf_play_outcome = RandomizedSearchCV(rf_play_outcome_search_model, rf_grid, cv=cv,random_state=42,n_iter=10,n_jobs=1,verbose=100, scoring=log_loss_scorer)
search_rf_play_outcome.fit(X_train,y_train,groups=group_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START bootstrap=False, max_depth=11, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=250
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
building tree 7 of 250
building tree 5 of 250
building tree 6 of 250
building tree 3 of 250
building tree 4 of 250
building tree 8 of 250
building tree 1 of 250
building tree 2 of 250
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.4min
building tree 9 of 250
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.5min
building tree 10 of 250
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  2.6min
building tree 11 of 250
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.6min
building tree 12 of 250
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.8min
building tree 13 of 250
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.8min
building tree 14 of 250
[Parallel(n_jobs=-1)]: Done   7 ta

In [ ]:
output_name = "drive_outcome_desc_basic"
mask_model = (
    (model_df.continuation==0)&
    (model_df.down!=0)&
    (model_df.play_counts==1)&
    (model_df.event_id.isin(scrimmage_plays_we_want))&
    (model_df[input_names+[output_name]].notna().all(axis=1))&
    (model_df["from_scrimmage"]==1)&
    (model_df["overtime"]==0)
)

X_train, y_train, group_train, X_test, y_test, group_test, X_val, y_val, group_val = create_train_test_val_df(model_df[mask_model], input_names, output_name)
cv=GroupKFold(n_splits=3)

log_loss_scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True)
rf_drive_outcome_search_model = RandomForestClassifier(verbose=100, n_jobs=-1, random_state=1)
search_rf_drive_outcome = RandomizedSearchCV(rf_drive_outcome_search_model, rf_grid, cv=cv,random_state=42,n_iter=10,n_jobs=1,verbose=100, scoring=log_loss_scorer)
search_rf_drive_outcome.fit(X_train,y_train,groups=group_train)


## Storing Predictions
* We'll add in the plays that didn't count
* The predictions are broken up by home/away team

In [ ]:
mask_model_predict = (
    (model_df.continuation==0)&
    (model_df.down!=0)&
    (model_df[input_names].notna().all(axis=1))&
    (model_df["from_scrimmage"]==1)&
    (model_df["overtime"]==0)
)

search_rf_play_class_names = ["search_rf_play_" + x for x in search_rf_play_outcome.classes_]
search_rf_drive_class_names = ["search_rf_drive_" + x for x in search_rf_drive_outcome.classes_]
model_df[search_rf_play_class_names] = pd.DataFrame(search_rf_play_outcome.predict_proba(model_df[mask_model_predict][input_names]), index=model_df[mask_model_predict].index)
model_df[search_rf_play_class_names] = model_df[search_rf_play_class_names].fillna(0)
model_df[search_rf_drive_class_names] = pd.DataFrame(search_rf_drive_outcome.predict_proba(model_df[mask_model_predict][input_names]), index=model_df[mask_model_predict].index)
model_df[search_rf_drive_class_names] = model_df[search_rf_drive_class_names].fillna(0)

search_rf_play_class_names_home = [x + "_home" for x in search_rf_play_class_names]
search_rf_play_class_names_away = [x + "_away" for x in search_rf_play_class_names]
search_rf_drive_class_names_home = [x + "_home" for x in search_rf_drive_class_names]
search_rf_drive_class_names_away = [x + "_away" for x in search_rf_drive_class_names]
model_df[search_rf_play_class_names_home] = model_df[search_rf_play_class_names].where(model_df.home_team_has_ball==1, 0)
model_df[search_rf_play_class_names_away] = model_df[search_rf_play_class_names].where(model_df.home_team_has_ball==0, 0)
model_df[search_rf_drive_class_names_home] = model_df[search_rf_drive_class_names].where(model_df.home_team_has_ball==1, 0)
model_df[search_rf_drive_class_names_away] = model_df[search_rf_drive_class_names].where(model_df.home_team_has_ball==0, 0)
display_html(model_df[mask_model][input_names + search_rf_play_class_names + search_rf_drive_class_names].head(15))
# display_html(model_df[mask_model][input_names + search_rf_drive_class_names].head(15))

### Score Difference Model
* Using the same inputs and adding the outputs of the previous model, we will predict the score differential probablities
* Since this model is an MLP model, we will normalize the inputs. (all features will be made so that the range is from 0 to 1)
* In this model, continuation, null values, end of quarters, and overtime is removed.
* NEW: spread and over/under are replaced by a weighted home and away vegas score prediction. The weight is based on time remaining normalized. So beginning of the game is the full score predictions, half time would be 0.5 * full score predictions

In [ ]:
model_df["away_vegas_score_pred"] = np.clip(model_df["cur_over_under"], 30, 80) * 0.5 + model_df["cur_spread"] * 0.5
model_df["home_vegas_score_pred"] = np.clip(model_df["cur_over_under"], 30, 80) * 0.5 - model_df["cur_spread"] * 0.5
model_df["away_vegas_score_pred_weighted"] = model_df["away_vegas_score_pred"] * (model_df["time_left_in_game"] / 3600)
model_df["home_vegas_score_pred_weighted"] = model_df["home_vegas_score_pred"] * (model_df["time_left_in_game"] / 3600)


# search_mlp_score_diff_clipped_rf_drive_preds = pickle.load(open(os.path.join(root_dir, "models/search_mlp_score_diff_clipped_rf_drive_preds.p"), 'rb'))
# search_mlp_score_diff_clipped_rf_drive_preds = pickle.load(open(os.path.join(root_dir, "models/search_mlp_score_diff_clipped_rf_drive_preds_vegas_adjusted.p"), 'rb'))
model_df["end_of_regulation_score_diff_change_clipped"] = np.clip(model_df["end_of_regulation_score_diff_change"], -35, 35)


# input_names_score_diff_pred = [item for item in input_names if item not in ["punt", "field_goal_attempt"]] + ["kick_off", "point_after_kick", "two_point_attempt"] + ["search_rf_play_first_down_home", "search_rf_play_first_down_away"] + search_rf_drive_class_names_home[1:] + search_rf_drive_class_names_away[1:]

input_names_score_diff_pred = [
    'time_left_in_half',
    'half',
    'current_score_diff',
    'current_score_total',
    # 'home_vegas_score_pred_weighted',
    # 'away_vegas_score_pred_weighted',
    'cur_spread',
    'cur_over_under',
    'home_timeouts_remaining',
    'away_timeouts_remaining',
    'ytg',
    'yd_from_goal',
    'down',
    'home_team_has_ball',
    'kick_off',
    'point_after_kick',
    'two_point_attempt',
    'search_rf_play_first_down_home',
    'search_rf_play_first_down_away',
    'search_rf_drive_field_goal_made_home',
    'search_rf_drive_field_goal_missed_home',
    'search_rf_drive_punt_home',
    'search_rf_drive_safety_home',
    'search_rf_drive_touch_down_home',
    'search_rf_drive_turnover_home',
    'search_rf_drive_turnover_on_downs_home',
    'search_rf_drive_field_goal_made_away',
    'search_rf_drive_field_goal_missed_away',
    'search_rf_drive_punt_away',
    'search_rf_drive_safety_away',
    'search_rf_drive_touch_down_away',
    'search_rf_drive_turnover_away',
    'search_rf_drive_turnover_on_downs_away'

]
output_name = "end_of_regulation_score_diff_change_clipped"

mask_model_score_diff = (
    (model_df.continuation==0)&
    (model_df[input_names_score_diff_pred+[output_name]].notna().all(axis=1))&
    ~(model_df.event_id.isin([12,57,58,13]))&
    (model_df["overtime"]==0)
)
X_train, y_train, group_train, X_test, y_test, group_test, X_val, y_val, group_val = create_train_test_val_df(model_df[mask_model], input_names_score_diff_pred, output_name)
mlp_grid = {
    'hidden_layer_sizes': [(50,50,50), (10,30,10),(100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
score_diff_change_list_clipped = list(model_df.end_of_regulation_score_diff_change_clipped.drop_duplicates().sort_values())

log_loss_scorer_score_diff = make_scorer(log_loss, greater_is_better=False, needs_proba=True, labels=score_diff_change_list_clipped)
mlp_score_diff_search_base = MLPClassifier(verbose=True, early_stopping=True, n_iter_no_change=5, random_state=1)
search_mlp_score_diff_clipped_rf_drive_preds = GridSearchCV(mlp_score_diff_search_base, mlp_grid, cv=cv,n_jobs=-1,verbose=100, scoring=log_loss_scorer_score_diff)
search_mlp_score_diff_clipped_rf_drive_preds.fit(X_train,y_train,groups=group_train)

normalized_score_pred_df = normalize_df(model_df[mask_model_score_diff][input_names_score_diff_pred], model_df[mask_model_score_diff & (model_df.season<2020)][input_names_score_diff_pred])
mlp_search_score_diff_clipped_rf_drive_preds_preds = pd.DataFrame(search_mlp_score_diff_clipped_rf_drive_preds.predict_proba(normalized_score_pred_df.values), index=model_df[mask_model_score_diff].index)
score_diff_clipped_rf_drive_preds_matrix = pd.DataFrame(np.zeros(mlp_search_score_diff_clipped_rf_drive_preds_preds.shape), index=mlp_search_score_diff_clipped_rf_drive_preds_preds.index)
score_diff_change_list_clipped = list(model_df.end_of_regulation_score_diff_change_clipped.drop_duplicates().sort_values())

for column in score_diff_clipped_rf_drive_preds_matrix.columns:
    score_diff_clipped_rf_drive_preds_matrix[column] = score_diff_change_list_clipped[column] + model_df["current_score_diff"]

model_df["xhome_win_mlp_search_clipped_rf_drive_preds"] = np.sum(mlp_search_score_diff_clipped_rf_drive_preds_preds.T[score_diff_clipped_rf_drive_preds_matrix.T>0], axis=0)
model_df["xovertime_mlp_search_clipped_rf_drive_preds"] = np.sum(mlp_search_score_diff_clipped_rf_drive_preds_preds.T[score_diff_clipped_rf_drive_preds_matrix.T==0], axis=0)
model_df["xaway_win_mlp_search_clipped_rf_drive_preds"] = np.sum(mlp_search_score_diff_clipped_rf_drive_preds_preds.T[score_diff_clipped_rf_drive_preds_matrix.T<0], axis=0)
model_df["xend_of_regulation_score_diff_mlp_search_clipped_rf_drive_preds"] = np.sum(score_diff_clipped_rf_drive_preds_matrix * mlp_search_score_diff_clipped_rf_drive_preds_preds, axis=1)

display_html(model_df[["xhome_win_mlp_search_clipped_rf_drive_preds", "xovertime_mlp_search_clipped_rf_drive_preds", "xaway_win_mlp_search_clipped_rf_drive_preds"]].dropna())
display_html(pd.DataFrame(mlp_search_score_diff_clipped_rf_drive_preds_preds.values, columns=score_diff_change_list_clipped))

In [ ]:
output_name = "end_of_regulation_score_total_diff_clipped"
input_names_total_score_pred = list(search_mlp_score_diff_clipped_rf_drive_preds.feature_names_in_) + ["xend_of_regulation_score_diff_change_mlp_search_clipped_rf_drive_preds"]
X_train, y_train, group_train, X_test, y_test, group_test, X_val, y_val, group_val = create_train_test_val_df(model_df[mask_model], input_names_total_score_pred, output_name)


mlp_total_score_search_base = MLPClassifier(verbose=True, early_stopping=True, n_iter_no_change=5, random_state=1)
search_mlp_total_score = GridSearchCV(mlp_total_score_search_base, mlp_grid, cv=cv,n_jobs=-1,verbose=100, scoring=log_loss_scorer)
search_mlp_total_score.fit(X_train,y_train,groups=group_train)
